Imports

In [3]:
import numpy as np
import matplotlib.pyplot as plt

### Method Set Up

The project description provides the following hint:

<span style="color:red;">

*For numerics, you could use something simple such as forward Euler combined with finite difference, but be sure to validate your choice of step size and other parameters.*

</span>


From this suggestion, we will start exploring the numerical solution with forward Euler.

##### Forward Euler Recap (diffusion only)
$$

U_m^{n + 1} = U_m^{n} + D\Delta t \frac{U_{m+1}^{n} - 2U_m^{n} + U_{m-1}^{n}}{(\Delta x )^2} \\ 

C = \frac{D\Delta t}{(\Delta x )^2} \\ 


U_m^{n + 1} = U_m^{n} + C[{U_{m+1}^{n} - 2U_m^{n} + U_{m-1}^{n}}] \\ 

$$

where $C < 0.5$ for stability.


Rewrite as matrix formulations (from Lecture Notes)

![](docs/forward_euler_matrix_formulation.png)



<!-- $
  \frac{U_m^{n + 1} - U_m^n}{\Delta t} = \frac{U_{m - 1}^n - 2 U_m^n + U_{m + 1}^n}{(\Delta x)^2} \quad \textrm{ for } m \in \left\{ 1, \ldots, M - 1 \right\}, n \in \left\{ 0, \ldots, N - 1 \right\},
$ -->

### Problem Set Up

#### Intravenous Injection

$$

C_t = D C_{xx} - v C_x \\

x \in (-\infty, \infty) \\

D = 0.1 m^2/s \\
v = 0.2 m/s \\

% U_m^{n + 1} = U_m^{n} + D\Delta t \frac{U_{m+1}^{n} - 2U_m^{n} + U_{m-1}^{n}}{(\Delta x )^2} \\ 
$$


#### Boundary Conditions

Infinite boundary conditions
$$
\lim_{x \to \infty} C(x, t) = 0 \\

\lim_{x \to -\infty} C(x, t) = 0 \\
$$


#### Conditions
At $t^*=60$ arrive at $h$: 

$60*0.2 = 12m$

Define conditions

In [6]:
D = 0.1 # Diffusion coefficient
v = 0.2 # Blood flow rate

Cf = 1e-3 # Desired final concentration

In [10]:
# Our zero dirichlet conditions

b0 = 0.0  # u(0,t)= b0
bL = 1.0  # u(L,t)= bL

#### Initial Concentration
let $t^* = t - 5$

$$
C(x, t^*=0) = \begin{cases} 
C_0, &  a \leq x \leq a+1 \\
0,  & \text{otherwise}
\end{cases}
$$




In [14]:
# def I(x): # initial u(x,0)
#     len_x = np.size(x)
#     i_x = 2.0 *np.ones(len_x) # the more complicated case
#     return i_x

def I(x): # initial u(x,0)
    return x

Solve using loops

In [15]:
# Set parameters
Nt_gaps = 20000    # number of timesteps
T = 1              # final time 
Nt_points = Nt_gaps + 1
t = np.linspace(0.,T,Nt_points)  # times at each time step

Nx_spaces = 100;   # number of spaces in x
L = 1; 
Nx_points = Nx_spaces + 1 
x = np.linspace(0, L, Nx_points)    # mesh points in space

dx = x[1] - x[0] 
dt = t[1] - t[0]


# Check our conditions
C = D*dt/(dx**2)
A = v*dt/(2*dx)

print("Delta x =", dx, "Delta t = ", dt, "C =", C, "A=", A)


Delta x = 0.01 Delta t =  5e-05 C = 0.05 A= 0.0005


In [16]:
# Solve
# Define the numerical solution 
# the first index is space and the second time
U = np.zeros((Nx_points,Nt_points))

# The initial condition
U[:,0] = I(x)

# enforce the boundary condition
U[0,0]  = b0 
U[-1,0] = bL

# Old/current values
u_old = I(x)

# Initialize new matrix
u = np.zeros(Nx_points)

for n in range(1,Nt_points):
    u[0] = b0
    u[-1] = bL
    
    # apply forward Euler on the internal points
    for i in range(1, Nx_points-1):
        u[i] = u_old[i] + C *(u_old[i-1] - 2*u_old[i] + u_old[i+1]) - A*(u_old[i+1] - u_old[i-1])

    # update u_old before next step
    u_old[:]= u
    # copy into full storage
    U[:,n] = u;


In [17]:
U.shape

(101, 20001)

Solve using matrices